# Projeto Final

Você fazem parte do time de Data Science e Analytics da Popolishoshop e receberam uma base de dados contendo as infromações sobre a última Black Friday que ocorreu. O time de negócio solicitou para vocês um relatório, onde especificaram algumas informações e também um estudo para que vocês respondessem utilizando as bases fornecidas.

Para esse desafio, vamos trabalhar com o data set [Black Friday](https://www.kaggle.com/mehdidag/black-friday), que reúne dados sobre transações de compras em uma loja de varejo. Esse dataset está quebrado em diferentes arquivos e é sua função entender como cada um se relaciona com o outro.

Vamos utilizá-lo para praticar a exploração utilizando pandas.

Na tabela a seguir podemos ver os nomes das colunas e as descrições dos campos.

| Coluna                 | Descrição                                                 |
|------------------------|-----------------------------------------------------------|
| User_ID                | ID do usuário                                             |
| Product_ID             | ID do produto                                             |
| Gender                 | Sexo do usuário                                           |
| Age                    | Ano em intervalos                                         |
| Occupation             | Ocupação (mascarada)                                      |
| City_Category          | Categoria da cidade (A, B, C)                             |
| StayInCurrentCityYears | Número de anos de permanência na cidade atual             |
| Marital_Status         | Estado civil                                              |
| ProductCategory1       | Categoria do produto (Mascarada)                          |
| ProductCategory2       | Categoria que o produto pode pertencer também (Mascarada) |
| ProductCategory3       | Categoria que o produto pode pertencer também (Mascarada) |
| Purchase               | Valor da compra                                           | 

Todo o código desenvolvido deve ser pensado para ser reutilizado. A avaliação se dará executando todo o notebook com outra tabela, de mesmas colunas. Sendo assim, pensem na qualidade e reprodução do código.

## _Set up_ da análise

Faça a leitura das três bases fornecidas e junte-as em um único DataFrame.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_user_profile = pd.read_csv("Enviar aos alunos/user_profile.csv", sep='|')
df_user_profile.head()

,User_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status
0,1000001,F,0-17,10,A,2,0
1,1000002,M,55+,16,C,4+,0
2,1000003,M,26-35,15,A,3,0
3,1000004,M,46-50,7,B,2,1
4,1000005,M,26-35,20,A,1,1


In [3]:
df_product_info = pd.read_csv("Enviar aos alunos/product_info.csv", sep=';')
df_product_info.head()

,Product_ID,Product_Category_1,Product_Category_2,Product_Category_3
0,P00069042,3,NaN,NaN
1,P00248942,1,6.0,14.0
2,P00087842,12,NaN,NaN
3,P00085442,12,14.0,NaN
4,P00285442,8,NaN,NaN


In [4]:
df_purchase = pd.read_csv("Enviar aos alunos/purchase.csv", sep=',')
df_purchase.head()

,User_ID,Product_ID,Purchase
0,1000001,P00069042,8370
1,1000001,P00248942,15200
2,1000001,P00087842,1422
3,1000001,P00085442,1057
4,1000002,P00285442,7969


In [5]:
df_bf = pd.merge(left=df_purchase, right=df_product_info, how='inner', on='Product_ID')
df_bf = pd.merge(left=df_bf, right=df_user_profile, how='inner', on='User_ID')
df_bf.shape

(537577, 12)

In [22]:
df = pd.read_csv('base_avaliacao.csv')

## Questão 1

Quantas observações e quantas colunas há no dataset completo (todas as bases juntas)? Responda no formato de uma tuple `(n_observacoes, n_colunas)`.

In [23]:
df_bf.shape

(1000, 12)

## Questão 2

Há quantas mulheres com idade entre 26 e 35 anos no dataset? Responda como um único escalar.

In [24]:
df_only_f = df_bf.drop_duplicates(subset=['User_ID'])
len(df_only_f[(df_only_f['Gender'] == 'F')&(df_only_f['Age'] == '26-35')])

77

## Questão 3

Quantos usuários únicos há no dataset? Responda como um único escalar.

In [25]:
df_bf['User_ID'].nunique()

831

## Questão 4

Qual porcentagem dos registros (percentual de linhas) possui ao menos um valor null (`None`, `ǸaN` etc)? Responda como um único escalar entre 0 e 1.

In [26]:
1-float(len(df_bf.dropna(how='any'))/len(df_bf))

0.719

## Questão 5

Quantos valores null existem na variável (coluna) com o maior número de null? Responda como um único escalar.

In [27]:
max(df_bf.isna().sum())

719

## Questão 6

Qual o valor mais frequente (sem contar nulls) em `Product_Category_3`? Responda como um único escalar.

In [28]:
int(df_bf['Product_Category_3'].value_counts().idxmax())

15

## Questão 7

Podemos afirmar que se uma observação é null em `Product_Category_2` ela também o é em `Product_Category_3`? Responda com um bool (`True`, `False`).

In [29]:
# Interseccao
c1 = df_bf[(df_bf['Product_Category_2'].isnull())&(df_bf['Product_Category_3'].isnull())]
# Singular
c2 = df_bf[df_bf['Product_Category_2'].isnull()]
c2['Product_Category_2'].equals(c1['Product_Category_3'])

True

## Questão 8

Qual o ID do usuário que mais gastou na Black Friday?

In [30]:
df_bf.groupby('User_ID')['Purchase'].sum().idxmax()

1001448

## Questão 9

Qual grupo (homens ou mulheres) mais gastou na Black Friday?

In [31]:
df_bf.groupby('Gender')['Purchase'].sum().idxmax()

'M'

## Questão 10

Faça uma nova tabela com a categoria mais comprada por cada cliente.

Obs: se ele comprou um produto que possuir valores nas três colunas de categorias, então deve-se considerar todas as categorias.

In [32]:
df_cat = df_bf[['User_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']]

In [33]:
df_flatten = df_cat.melt(id_vars=['User_ID'],value_name='category').drop(columns=['variable'])
df_flatten = df_flatten.dropna()
df_flatten['inc'] = 1

In [34]:
pd.pivot_table(df_flatten, index='User_ID', columns='category', values='inc', fill_value=0, aggfunc=sum)

category,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0
User_ID,,,,,,,,,,,,,,,,,,
1000007,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1000010,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
1000011,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
1000014,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
1000023,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006005,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1006006,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1006016,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0


## Questão 11

Normalize a coluna Purchase. A fórmula de normalização é:


$$x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}} $$

In [35]:
df_bf['purchase_normalized'] = (df_bf['Purchase'] - df_bf['Purchase'].min()) / (df_bf['Purchase'].max() - df_bf['Purchase'].min())
df_bf['purchase_normalized']

0      0.361904
1      0.150036
2      0.082879
3      0.287604
4      0.411732
         ...   
995    0.638899
996    0.290182
997    0.329403
998    0.138413
999    0.415959
Name: purchase_normalized, Length: 1000, dtype: float64

## Questão 12
O estado civil influencia no valor gasto e na categoria de produto comprada? Mostre!

Se eu quisesse vender mais produtos da categoria 14, deveria investir em propagandas para qual estado civil?

## Questão 13
Quais as variáveis que mais impactaram no valor da compra? Como você chegou a essa conclusão?